# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [13]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'D:/data/Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head(15)

In [15]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [16]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head(10)

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
5,3,NaN,0,0,8.4583
6,1,54.0,0,0,51.8625
7,3,2.0,3,1,21.0750
8,3,27.0,0,2,11.1333
9,2,14.0,1,0,30.0708


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [17]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6981796497395015

In [18]:
# 空值補 0, and logistic regression
df_m2 = df.fillna(0)
train_X = df_m2[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
# 空值補 0 的預測結果相對較好

0.6992969681752557

In [19]:
# 空值補平均值
df_m3 = df.fillna(df.mean())
train_X = df_m3[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6947900320130562

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [24]:
"""
Your Code Here
"""
# 空值補 0, 未標準化
df2 = df.fillna(0)
train_X = df2[:train_num]
estimator = LogisticRegression()
scores1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 0, 搭配最大最小化
df3 = MinMaxScaler().fit_transform(df2)
train_X = df3[:train_num]
estimator = LogisticRegression()
scores2 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 0, 搭配標準化
df4 = StandardScaler().fit_transform(df2)
train_X = df4[:train_num]
estimator = LogisticRegression()
scores3 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print(scores1)
print(scores2)
print(scores3)
#以最大最小值標準化的方式相對來說預測較準

0.6992969681752557
0.7015378821166279
0.6981733726696377


In [27]:
max([scores1, scores2, scores3])

0.7015378821166279